In [8]:
from google.cloud import storage
import json
from dateutil.parser import parse
import pandas as pd

In [2]:
storage_client = storage.Client()
bucket = storage_client.bucket('dotufp-raw')
blob_name = 'twitter_home_timeline/dotufp/2019-11-27T07:00:02Z.raw'
collected_timestamp = '2019-11-27T07:00:02Z'
blob = bucket.blob(blob_name)
raw_str = blob.download_as_string()

raw_str[:1000]

/home/dylan/anaconda3/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/home/dylan/anaconda3/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/au

b'[{"created_at":"Wed Nov 27 06:59:32 +0000 2019","id":1199583516561817600,"id_str":"1199583516561817600","full_text":"RT @CharlotteAlter: Pete called the writer who wrote that viral essay, and @michaelharriot  wrote a follow-up about their conversation. It\'\\u2026","truncated":false,"display_text_range":[0,140],"entities":{"hashtags":[],"symbols":[],"user_mentions":[{"screen_name":"CharlotteAlter","name":"Charlotte Alter","id":774510134,"id_str":"774510134","indices":[3,18]},{"screen_name":"michaelharriot","name":"michaelharriot","id":96149284,"id_str":"96149284","indices":[75,90]}],"urls":[]},"source":"\\u003ca href=\\"https:\\/\\/twitterrific.com\\/ios\\" rel=\\"nofollow\\"\\u003eTwitterrific for iOS\\u003c\\/a\\u003e","in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":715073,"id_str":"715073","name":"Alpha Chen","screen_name":"kejadlen","location":"","description":"he\\

In [3]:
raw_json = json.loads(raw_str)

len(raw_json), raw_json[0].keys()

(100,
 dict_keys(['created_at', 'id', 'id_str', 'full_text', 'truncated', 'display_text_range', 'entities', 'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'retweeted_status', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'lang']))

In [4]:
raw_json[1]

{'created_at': 'Wed Nov 27 06:44:37 +0000 2019',
 'id': 1199579762009788416,
 'id_str': '1199579762009788416',
 'full_text': 'You chitter at the other raccoon. They chitter back. Their name is Zorro. They reach down to pull you up onto the top of the fire engine.\n\nYou are on top of a fire engine. Zorro the raccoon is here. You see a ladder and a firefighter helmet.\n\nYour whiskers have cream on them.',
 'truncated': False,
 'display_text_range': [0, 276],
 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []},
 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>',
 'in_reply_to_status_id': 1199492065958277120,
 'in_reply_to_status_id_str': '1199492065958277120',
 'in_reply_to_user_id': 21604764,
 'in_reply_to_user_id_str': '21604764',
 'in_reply_to_screen_name': 'JoeSondow',
 'user': {'id': 21604764,
  'id_str': '21604764',
  'name': 'Joe Sondow',
  'screen_name': 'JoeSondow',
  'location': 'Bay Area, California',
  'description'

In [9]:
tweets = []

for i in range(10):
    raw_tweet = raw_json[i]
    tweet = {}

    tweet['collected_timestamp'] = collected_timestamp
    tweet['text'] = raw_tweet.get('full_text') or raw_json[0].get('text')
    tweet['id'] = raw_tweet['id']
    tweet['created_at'] = parse(raw_tweet['created_at']).strftime('%Y-%m-%dT%H:%M:%SZ')
    tweet['user_id'] = raw_tweet['user']['id']
    tweet['user_screen_name'] = raw_tweet['user']['screen_name']
    tweet['user_followers_count'] = raw_tweet['user']['followers_count']
    tweet['retweet_count'] = raw_tweet['retweet_count']
    tweet['favorite_count'] = raw_tweet['favorite_count']

    tweets.append(tweet)
    
tweets_df = pd.DataFrame(tweets)

In [6]:
tweet

{'collected_timestamp': '2019-11-27T07:00:02Z',
 'text': "RT @AccessibleProd: @GailSimone Gail... I have around $9,000 left to be debt free, I'm in need of help &amp; I feel like giving up on everythin…",
 'id': 1199575857167130624,
 'created_at': '2019-11-27T06:29:06Z',
 'user_id': 16024323,
 'user_screen_name': 'GailSimone',
 'user_followers_count': 145619,
 'retweet_count': 5,
 'favorite_count': 0}

In [10]:
tweets_df

,collected_timestamp,text,id,created_at,user_id,user_screen_name,user_followers_count,retweet_count,favorite_count
0,2019-11-27T07:00:02Z,RT @CharlotteAlter: Pete called the writer who...,1199583516561817600,2019-11-27T06:59:32Z,715073,kejadlen,589,576,0
1,2019-11-27T07:00:02Z,You chitter at the other raccoon. They chitter...,1199579762009788416,2019-11-27T06:44:37Z,21604764,JoeSondow,9387,7,17
2,2019-11-27T07:00:02Z,◽◽◽👔◽◽◽\n◽◽◽👔👔◽◽ Next\n◽◽◽👔◽◽◽ 👘\n◽◽◽◽◽◽◽ 👘\n◽...,1199579299856048128,2019-11-27T06:42:46Z,842095599100997636,EmojiTetra,13923,0,2
3,2019-11-27T07:00:02Z,RT @tayziken: one of the funniest things in th...,1199577680749846530,2019-11-27T06:36:20Z,14794913,AthertonKD,27016,13712,0
4,2019-11-27T07:00:02Z,draft dump — women with potatoes edition https...,1199577198392274946,2019-11-27T06:34:25Z,51752499,House_Feminist,32165,0,16
5,2019-11-27T07:00:02Z,RT @jtag10: star trek’s continual return to ea...,1199576655884218368,2019-11-27T06:32:16Z,14794913,AthertonKD,27016,542,0
6,2019-11-27T07:00:02Z,RT @emontoya42: I'm starting to think Gail is ...,1199576493879222272,2019-11-27T06:31:37Z,16024323,GailSimone,145619,1,0
7,2019-11-27T07:00:02Z,RT @BuildSoil: Imagine Carbon is kids at a par...,1199576271291764736,2019-11-27T06:30:44Z,14794913,AthertonKD,27016,9,0
8,2019-11-27T07:00:02Z,RT @BuildSoil: Active Storage means the few ho...,1199576234843197441,2019-11-27T06:30:36Z,14794913,AthertonKD,27016,6,0
9,2019-11-27T07:00:02Z,RT @AccessibleProd: @GailSimone Gail... I have...,1199575857167130624,2019-11-27T06:29:06Z,16024323,GailSimone,145619,5,0
